## Plaicraft: Predicting Played Hours from Player Type

In [1]:
import numpy as np
import pandas as pd
import scipy as sci
import matplotlib.pyplot as plt
import os
import altair as alt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn import set_config
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split


# alt.data_transformers.enable('vegafusion')
# alt.renderers.enable('default')
# set_config(transform_output="pandas")

print("packages imported")

packages imported


# (1) Introduction:

## 1. Background Information:

The Pacific Laboratory for Artificial Intelligence (PLAI) at UBC is working on a project to study human behavior in video game environments using generative AI and machine learning. A key component of this research involves Plaicraft.ai, which is a public platform that allows players to engage with Minecraft in a cloud environment so that they can collect valuable gameplay data. They aim to collect over 10,000 hours of gameplay to advance artificial general intelligence (AGI) research. They need our help to understand ***which "kinds" of players are most likely to contribute a large amount of data, and how can this knowledge improve recruitment efforts*** answering this question will enable PLAI to target specific player profiles for recruitment and ensure they have efficient resource allocation for software.

### 2. Exploring Datasets

Our report addresses this question by analyzing two datasets: one about the individual player information (players.csv) and another detailing gameplay sessions (session.csv). Through data wrangling, visualization, and predictive modeling, the report aims to identify characteristics of high-contributing players and provide actionable insights for PLAI’s recruitment strategies. This analysis is not only significant for optimizing the current project but also sets a precedent for future data-driven research in gaming and AI.


#### <u>Players.csv</u>:

- Total Observations (Rows): 196
- Total Variables (Columns): 9

Dataset includes includes the following columns:

| Column Name        | Data Type    | Description|
---------------------|--------------|-------------|
| `experience`       | Categorical  | Skill level of the player (e.g., Beginner, Veteran, Pro).          
| `subscribe`        | Boolean      | Shows if a player is subscribed to the service.                                            
| `hashedEmail`      | String       | Unique identifier for each player.                                                    
| `played_hours`     | Float        | Total hours the player has engaged in the game.      
| `name`             | String       | First name of the player.                                                                    
| `gender`           | Categorical  | Gender of the player (Male, Female etc)                                      
| `age`              | Integer      | Age of the player.                                                                          
| `individualId`     | String       | Additional unique identifier for each player      
| `organizationName` | String       | Name of the organization the player is associated with.    

##### Potential Issues:

| Column Name        | Potential Issues                             |
|---------------------|----------------------------------------------|
| `experience`       | Could be difficult to directly code with this as we would somehow have to represent that Beginner < Amateur < Pro < Veteran |
| `subscribe`        | Not any apparent issues        |
| `hashedEmail`      | Irrelevant for all types of data analysis and should be excluded. |
| `played_hours`     | Potential outliers can effect the model (eg. 218 hours for one player) |
| `name`             | Not relevant for data analysis        |
| `gender`           | Will have to think about how to handling cases of "Prefer not to Say"  |
| `age`              | Some unrealistic values (e.g. 91).    |
| `individualId`     | All values are NaaN so should be excluded        |
| `organizationName` | All values are NaaN so should be excluded       |


Lastly it is important to note that there are potential issues with the way the data was collected:

1. Dataset only includes players who chose to participate or were actively recruited by the research team so it might not represent the entire player population accurately.
   
2. Some variables like played_hours are likely tracked automatically by the server, which does not account for players who leave their accounts logged in without active play.

#### <u>Sessions.csv:</u>:

- Total Observations (Rows): 1,535
- Total Variables (Columns): 5

##### Dataset includes the following columns:

| Column Name           | Data Type    | Description                                                                                   |
|------------------------|--------------|-----------------------------------------------------------------------------------------------|
| `hashedEmail`          | String       | Unique hashed identifier for each player.                       |
| `start_time`           | String       | Recorded start time of the session in a date-time format.          |
| `end_time`             | String       | Recorded end time of the session in a date-time format.       |
| `original_start_time`  | Float        | Timestamp representation of the `start_time` column.     |
| `original_end_time`    | Float        | Timestamp representation of the `end_time` column.          |


##### Potential Issues:

| Column Name           | Potential Issues                                                                                   |
|------------------------|----------------------------------------------------------------------------------------------------|
| `hashedEmail`          | Not directly useful for analysis; serves only as a unique identifier.                             |
| `start_time`           | May need to convert to a proper datetime format for analysis.                                     |
| `end_time`             | Missing values in some rows; handling these gaps will be necessary.                                  |
| `original_start_time`  | Redundant as it duplicates information from `start_time`.                                          |
| `original_end_time`    | Redundant as it duplicates information from `end_time`.          |

#### Observations and Insights:
1. Both `original_start_time` and `original_end_time` columns are timestamps, but they provide the same information as `start_time` and `end_time` we only need one for the analysis
   
2. Some missing `end_time` that will need to be properly handled.

3. In order to calculate session duration we will need to convert `start_time` and `end_time`.

4. Similar to the `players.csv` dataset, information may be biased as it includes only active players, which may not represent the broader population of potential players.

# (2) Methods & Results:

> describe the methods you used to perform your analysis from beginning to end that narrates the analysis code.

K-NN Regression is one way to predict using `age` because `played_hours` is a quantitative variable, not qualitative, and does not make assumptions about data linearity.

Model Selection:
* For K-NN-Regression, 70%-30% train-test split with a set random seed.
* 5-fold cross-validation to select K with the lowest cross-validation root mean squared prediction error (RMSPE).
* Fit the training data
* Predict with the testing data
* Calculate the model's RMSPE
* Conduct simple linear regression and multivariable linear regression to calculate RMSPE
* Comparing RMSPE of the 3 models: selecting the model with the lowest test dataset RMSPE

DONE:
> loads data
> 
> wrangles and cleans the data to the format necessary for the planned analysis

TO-DO:
> performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis
> 
> creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
> 
> performs the data analysis
>
> creates a visualization of the analysis

(Steph's findings:)
- Most participants are 15-25 years old (around 70+95=165 participants). K-NN may not predict well in other ages with fewer or no data points.
-A few younger players (ages 10-30) contributed large hours (20+), but most individuals contributed <10 hours.
-Many players play 0-1 hours. Researchers should define a "large" amount of data (eg. >1 hour)
-As age increases, the average `played_hours` fluctuates, suggesting a non-linear pattern with `age`.


> note: all figures should have a figure number and a legend
>


### Load data

In [2]:
#import players.csv dataframe as URL
url_players = "https://drive.google.com/uc?id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz"
players = pd.read_csv(url_players)

players.head()

experience  subscribe                                        hashedEmail  \
0        Pro       True  f6daba428a5e19a3d47574858c13550499be23603422e6...   
1    Veteran       True  f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...   
2    Veteran      False  b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...   
3    Amateur       True  23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...   
4    Regular       True  7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...   

   played_hours       name  gender  age  individualId  organizationName  
0          30.3     Morgan    Male    9           NaN               NaN  
1           3.8  Christian    Male   17           NaN               NaN  
2           0.0      Blake    Male   17           NaN               NaN  
3           0.7      Flora  Female   21           NaN               NaN  
4           0.1      Kylie    Male   21           NaN               NaN

In [3]:
#import sessions.csv dataframe as URL
url_sessions = "https://drive.google.com/uc?id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB"
sessions = pd.read_csv(url_sessions)

sessions.head()

hashedEmail        start_time  \
0  bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...  30/06/2024 18:12   
1  36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...  17/06/2024 23:33   
2  f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...  25/07/2024 17:34   
3  bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...  25/07/2024 03:22   
4  36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...  25/05/2024 16:01   

           end_time  original_start_time  original_end_time  
0  30/06/2024 18:24         1.719770e+12       1.719770e+12  
1  17/06/2024 23:46         1.718670e+12       1.718670e+12  
2  25/07/2024 17:57         1.721930e+12       1.721930e+12  
3  25/07/2024 03:58         1.721880e+12       1.721880e+12  
4  25/05/2024 16:12         1.716650e+12       1.716650e+12

### Wrangle Data

From `players.csv`
- `age` is already present, so no additional wrangling is required.
- `experience` was converted from qualitative to quantitative `experience_val` by assigning the experience level from 1 (least experienced) to 5 (most experienced). This makes the assumption that there are equal changes in experience level between each value, which is not true.

From `sessions.csv`, `hashedEmail` was used as an identifier to link session data to each player.

- `start_time` and `end_time` were subtracted to calculate `session_length`
- `subscribe` was assigned a number where True = 1 and False = 0


In [4]:
#define experience level as numbers 1 to 5
experience_mapping = {
    'Beginner': 1,
    'Amateur': 2,
    'Regular': 3,
    'Veteran': 4,
    'Pro': 5
}

#convert dates to datetime object
sessions['start_time'] = pd.to_datetime(sessions['start_time'], format="%d/%m/%Y %H:%M")
sessions['end_time'] = pd.to_datetime(sessions['end_time'], format="%d/%m/%Y %H:%M")

#calculate session length
sessions['session_length'] = (sessions['end_time'] - sessions['start_time']).dt.total_seconds() / 3600

#group data in sessions dataframe based on player email
player_sessions = sessions.groupby('hashedEmail').agg(
    number_sessions=('session_length', 'size'), #calculate total number of sessions per player
    mean_session_length=('session_length', 'mean'), #calculate average session length per player
    std_session_length=('session_length', 'std') #calculate standard deviation (std) of session length per player
    #note: session length (mean and std) were unused in our analysis
    #      but it could be useful for the client to answer other questions
).reset_index()

#combine player and session data using email
players_combined = pd.merge(players, player_sessions, on='hashedEmail', how='left')

#convert experience into numbers 1 to 5
players_combined['experience_val'] = players_combined['experience'].map(experience_mapping)

#convert subscribe into binary
players_combined['subscribe_binary'] = players_combined['subscribe'].astype(int)

#drop data missing for variables of interest - assume it is missing at random
players_combined = players_combined.dropna(subset=['subscribe_binary', 'experience_val', 'age', 'number_sessions', 'played_hours'])
players_combined.head(5)

experience  subscribe                                        hashedEmail  \
0        Pro       True  f6daba428a5e19a3d47574858c13550499be23603422e6...   
1    Veteran       True  f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...   
2    Veteran      False  b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...   
3    Amateur       True  23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...   
4    Regular       True  7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...   

   played_hours       name  gender  age  individualId  organizationName  \
0          30.3     Morgan    Male    9           NaN               NaN   
1           3.8  Christian    Male   17           NaN               NaN   
2           0.0      Blake    Male   17           NaN               NaN   
3           0.7      Flora  Female   21           NaN               NaN   
4           0.1      Kylie    Male   21           NaN               NaN   

   number_sessions  mean_session_length  std_session_length  experience_val  \
0             27.0             1.246296            0.902162               5   
1              3.0             1.416667            1.233671               4   
2              1.0             0.083333                 NaN               4   
3              1.0             0.833333                 NaN               2   
4              1.0             0.150000                 NaN               3   

   subscribe_binary  
0                 1  
1                 1  
2                 0  
3                 1  
4                 1

### Summary of Dataset and Visualizations

In [5]:
# summarize the dataset - mean age etc.

In [6]:
#number of players in each age group
players_age = (
    alt.Chart(players, title="Number of Players per Age in Plaicraft").mark_bar().encode(
        x = alt.X("age").title("Age (years)").bin(maxbins=20),
        y = alt.Y("count()").title("Number of Players")
    )
    .configure_axis(titleFontSize=12)
)

players_age

alt.Chart(...)

*Figure 1: Histogram of age (years) versus number of players.*

Most participants are 15-25 years old (around 70+95=165 participants). K-NN may not predict well in other ages with fewer or no data points.

In [7]:
#overplotted but general overview of age and hours played
players_scatterplot = (
    alt.Chart(players, title="Hours Contributed by Each Player with Age in Plaicraft")
    .mark_circle(size=60, opacity=0.40)
    .encode(
        x = alt.X("age").title("Age (years)"),
        y = alt.Y("played_hours").title("Total Time Played (hours)")
    )
    .configure_axis(titleFontSize=12)
)

players_scatterplot

alt.Chart(...)

*Figure 2.1: Scatterplot of age (years) versus total time played (hours).*

A few younger players (ages 10-30) contributed large hours (20+), but most individuals contributed <10 hours (in the overplotted area).

In [8]:
#zoom in on Hours 0-8
minimum_hours = 0
maximum_hours = 8

players_played = (
    players[
        (players["played_hours"] >= minimum_hours) &
        (players["played_hours"] <= maximum_hours)
    ]
)

players_scatterplot = (
    alt.Chart(players_played, 
              title="Total Player Time per Age and Experience between 0-8 hours in Plaicraft")
    .mark_point(opacity=0.50)
    .encode(
        x = alt.X("age").title("Age (years)"), #.scale(type="log")
        y = alt.Y("played_hours")
            .title("Total Time Played (hours)")
            .scale(domain=[minimum_hours, maximum_hours]),
        color = alt.Color("experience").title("Experience Level")
    )
    .configure_axis(titleFontSize=12)
)
players_scatterplot

alt.Chart(...)

*Figure 2.2: Scatterplot of age (years) versus total time played (hours).*

Many players play 0-1 hours. Researchers should define a "large" amount of data (eg. >1 hour). There is a wide range of experiences levels across age.

In [9]:
#picturing relationship between 'number of played hours' and 'subscribe'
hour_vs_subscribe = (
    alt.Chart(players_combined, title="Total Played Hours with Subscription Status").mark_bar().encode(
        x = alt.X("subscribe").title("Subscription Status"),
        y = alt.Y("played_hours").title("Total Time Played (hours)")
    )
    .configure_axis(titleFontSize=12)
)

hour_vs_subscribe

alt.Chart(...)

*Figure 3: Bar plot with Subscription Status (True/False) and Total Time Player (hours)*

More hours were contributed by players subscribed to Plaicraft's emailing system.

In [10]:
#picturing relationship between 'number of played hours' and 'number of sessions'

played_hour_vs_subscribe = (
    alt.Chart(players_combined, title="Played hours vs subscription").mark_bar().encode(
        x = alt.X("number_sessions").title("Number of Sessions").bin(maxbins=20),
        y = alt.Y("played_hours").title("Total Time Played (hours)"),
    )
    .configure_axis(titleFontSize=12)
)

played_hour_vs_subscribe

alt.Chart(...)

*Figure 4: Distribution of Number sessions and Total Time Player (hours)*

Total time played increases then decreases as the number of sessions increases, presenting a non-linear relationship. Participants who played 140-160 sessions contributed the most played hours.

In [11]:
#splitting the testing and training data with ratio 7:3
players_training, players_testing = train_test_split(
   players_combined, test_size=0.3, random_state=2000 
)

X_train = players_training[["age"]]
y_train = players_training['played_hours']

X_test = players_testing[["age"]] 
y_test = players_testing['played_hours'] 

#make a pipeline for our model
players_pipe = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor()
)

#making grid parameters
param_grid = {'kneighborsregressor__n_neighbors': range(1, 30)}

#doing cross validation
player_tuned = GridSearchCV(
    players_pipe,
    param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

# #making a new dataframe from the results of our cross validation
# player_results = pd.DataFrame(
#     player_tuned.fit(X_train, y_train).cv_results_
# )

#finding best number of K
player_min = player_tuned.best_params_
player_best_RMSPE = -player_tuned.best_score_

player_min

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
#using our best K in the model
k = 28
knn_model = KNeighborsRegressor(n_neighbors=k)
knn_model.fit(X_train, y_train)

#predicting with our testing dataframe
y_pred = knn_model.predict(X_test)

#finding RM
mse = mean_squared_error(y_test, y_pred)
rmspe = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Prediction Error (RMSPE):", rmspe)
print("R-squared:", r2)

In [ ]:
#plotting the predictions and the actual data 
plot_data = pd.DataFrame({
    'Age': X_test.squeeze(),
    'Actual Played Hours': y_test,
    'Predicted Played Hours': y_pred
})

scatter = alt.Chart(plot_data).mark_circle(size=60).encode(
    x=alt.X('Age', title='Age'),
    y=alt.Y('Actual Played Hours', title='Played Hours'),
    tooltip=['Age', 'Actual Played Hours', 'Predicted Played Hours']
).properties(
    title=f'k-NN Regression: Age vs Played Hours (K={k})'
)

line = alt.Chart(plot_data).mark_line(color='red').encode(
    x='Age',
    y='Predicted Played Hours'
)

final_plot = scatter + line

final_plot.show()

From the plot above, we can see that most of our data comes from the age range 15-28, and the hours played mostly below 10. However, we can see that there are puliers where someone near 100 years old plays the game and datapoint at player hours nearing 180. 

Our model predicted that the most played hours would come from players aged ~18-27, which seems consistent with our real data described before. However, we can also see that there are no data from the age range 33-49 and 51-99, yet our model still predicted an amount of played hours. Therefore we know that our model predicts poorly on age groups with no data points. 

## Attempt for a simple linear regression using age as covariate and played_hour as responsible variable:

In [ ]:
#making a new model for linear regression and fit the training data
model = LinearRegression()
model.fit(X_train, y_train)

#prediciting y pred with our testing data and new linear regression model
y_pred = model.predict(X_test)

#finding RMSPE
mse = mean_squared_error(y_test, y_pred)
rmspe = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
n = X_test.shape[0]
p = X_test.shape[1]
adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

intercept = model.intercept_
slope = model.coef_[0]


print("Mean Squared Error:", mse)
print("Root Mean Squared Prediction Error (RMSPE):", rmspe)
print("R-squared:", r2)
print("Adjusted R-squared:", adjusted_r2)
print("Intercept:", model.intercept_)
print("Coefficient:", model.coef_)

print("Intercept (b0):", intercept)
print("Slope (b1):", slope)

In [ ]:
#plotting results of linear regression

X_all = pd.concat([X_train, X_test], axis=0)
y_all = pd.concat([y_train, y_test], axis=0)

X_all['Actual'] = y_all
X_all['Predicted'] = model.predict(X_all[['age']])

scatter = alt.Chart(X_all.reset_index()).mark_circle(size=60).encode(
    x=alt.X('age', title='Age'),
    y=alt.Y('Actual', title='Played Hours'),
    tooltip=['age', 'Actual', 'Predicted']
).properties(
    title="Linear Regression: Played Hours vs Age (All Data)"
)

line = alt.Chart(X_all.reset_index()).mark_line(color='red', size=2).encode(
    x=alt.X('age', title='Age'),
    y=alt.Y('Predicted', title='Played Hours')
)

plot = scatter + line
plot.display()

The plot above shows a decreasing trend from the prediction made by our model.

In [ ]:
X = players_combined[['subscribe_binary', 'experience_val', 'age', 'number_sessions']]
y = players_combined['played_hours']

X_train, X_test = train_test_split(X, test_size=0.3, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42)


model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
n = X_test.shape[0]
p = X_test.shape[1]
adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

print("Mean Squared Error:", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared:", r2)
print("Adjusted R-squared:", adjusted_r2)
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

coefficients = pd.DataFrame({
    'Feature': ['subscribe_binary', 'experience_val', 'age', 'number_sessions'],
    'Coefficient': model.coef_
})

print(coefficients)